Purpose: Plan of whole pipeline from Ref-genome/vcf-file/TSS-file/ to basenji-output

Todo:
ask Ziga about: vcf-gtex-gile
run libs through with him

add warning if Ref-Base is not base in VCF -> probl. due to shifted positions

Classes: 

    [later: PipelineConfigurator: takes Gene-IDs  and sample-IDs from user, writes into output file
            Calls all others, needs therefor all files. Makes sure that Intervall-Translator and model are compatible
    ]
    TODO:
    Basenji-call with input
    ResultWriter: puts results in Xarray with dimensions Gene/Sample/Outputvalues, stores it to disk
    
    Now: REPDataloader which reads in TSS, passes intervals to vcfFastaExtractor:
    
    TSS: not dataloader, is small...
    but: IntervalCalculator: do...
    
    vcfFastaExtractor
    random fasta access: pyfaidx
    random vcf-access: pyVCF
    
    output: one-hot-encoded seq with annotations: geneID, sampleID, posiRefStart , ALLELE (0 or 1)
    
Input-files:

   RefGenome in fasta: https://raw.githubusercontent.com/kipoi/kipoiseq/master/tests/data/hg38_chr22_32000000_32300000.fa
   
   (not used: intervalls-example: https://raw.githubusercontent.com/kipoi/kipoiseq/master/tests/data/intervals_51bp.tsv)
   
   CAGE TSS `/s/project/avsec/ExPecto/resources/geneanno.csv`
   has:
    lincRNA
    lincRNA
    protein_coding
    rRNA
   -> which to use?
   
   GTEx-vcf? maybe from /s/project/gtex-processed/vcf/GTEx_Analysis_20150112_WholeGenomeSeq_148Indiv_GATK_HaplotypeCaller.vcf.gz
   
   RefGenome: /s/genomes/human/hg38$ 



   
   
------
   Use loading-param from:
   https://github.com/kipoi/kipoi/blob/master/kipoi/data.py
          dl = DataLoader(self, batch_size=1,
                        collate_fn=numpy_collate_concat,    <---
                        shuffle=False,
                        num_workers=num_workers,
drop_last=False)
    

In [ ]:
Todo next:      
    
    buffer zarr-writer...

    
    
    test writer some more
    
    prep env for gpu01 
        (write down my env here) + #pip install tensorflow-gpu==1.5.0
        use env bazenji-pred (basenji-pred file in folder)
            compare to kipoi-manuscript-tlearn-gpu:
              conda list | grep cu
              conda list | grep tensor

        
        export CUDA_VISIBLE_DEVICES=0 # set this before running stuff!
    
    get full script of workflow 
    
    prep vcf file -) chrX everywhere
    
    igrnoe */* -> test what number they have...
    
    measure perf: gpu at 100%?
        watch nvidia-smi and htop
    if below 100%: use more workers / batch-writer from ziga 
    

# Playing aroudn with fasta/vcf/tss-libs and fileformats

In [4]:
 !pip install pyfaidx

You are using pip version 19.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [85]:
from pyfaidx import Fasta
refGenome = Fasta('/s/genomes/human/hg38/hg38.fa')

Exception ignored in: <bound method BaseExtractor.__del__ of <kipoiseq.extractors.base.FastaStringExtractor object at 0x7f3184cb1f98>>
Traceback (most recent call last):
  File "/opt/modules/i12g/anaconda/3-5.0.1/envs/rep/lib/python3.6/site-packages/kipoiseq/extractors/base.py", line 16, in __del__
    return self.close()
  File "/opt/modules/i12g/anaconda/3-5.0.1/envs/rep/lib/python3.6/site-packages/kipoiseq/extractors/base.py", line 59, in close
    return self.fasta.close()
AttributeError: 'FastaStringExtractor' object has no attribute 'fasta'


In [86]:
refGenome["chr1"][40000:40100]

>chr1:40001-40100
gcctcatggaggggatcagctgcgaggagctaagagccccctccagtcgatgctcaccaggaagctgaggtcttgtgtccagcaccctgcatagaactga

In [83]:
import vcf

vcf_reader = vcf.Reader(filename='/s/project/gtex-processed/vcf/GTEx_Analysis_20150112_WholeGenomeSeq_148Indiv_GATK_HaplotypeCaller.vcf.gz')

Exception ignored in: <bound method BaseExtractor.__del__ of <kipoiseq.extractors.base.FastaStringExtractor object at 0x7f321ec61438>>
Traceback (most recent call last):
  File "/opt/modules/i12g/anaconda/3-5.0.1/envs/rep/lib/python3.6/site-packages/kipoiseq/extractors/base.py", line 16, in __del__
    return self.close()
  File "/opt/modules/i12g/anaconda/3-5.0.1/envs/rep/lib/python3.6/site-packages/kipoiseq/extractors/base.py", line 59, in close
    return self.fasta.close()
AttributeError: 'FastaStringExtractor' object has no attribute 'fasta'


In [84]:
#example for calling
for record in vcf_reader.fetch('20', 1110695, 1110780):  
    print(record.samples)

[Call(sample=GTEX-QV31-0003-SM-5URDH, CallData(GT=0/0, AD=[35, 0], DP=35, GQ=79, PL=[0, 79, 1169])), Call(sample=GTEX-R55E-0003-SM-5URBX, CallData(GT=0/0, AD=[27, 0], DP=27, GQ=69, PL=[0, 69, 847])), Call(sample=GTEX-X4XY-0002-SM-5URBG, CallData(GT=0/0, AD=[34, 0], DP=34, GQ=62, PL=[0, 62, 949])), Call(sample=GTEX-X62O-0001-SM-5URD5, CallData(GT=1/1, AD=[0, 55], DP=55, GQ=99, PL=[1645, 164, 0])), Call(sample=GTEX-N7MS-0009-SM-5JK3E, CallData(GT=0/0, AD=[23, 0], DP=23, GQ=60, PL=[0, 60, 900])), Call(sample=GTEX-N7MT-0009-SM-5JK4X, CallData(GT=0/1, AD=[21, 12], DP=33, GQ=99, PL=[324, 0, 619])), Call(sample=GTEX-NFK9-0004-SM-5JK4Y, CallData(GT=0/0, AD=[27, 0], DP=27, GQ=73, PL=[0, 73, 924])), Call(sample=GTEX-NL3H-0009-SM-5JK37, CallData(GT=0/0, AD=[29, 0], DP=29, GQ=67, PL=[0, 67, 1016])), Call(sample=GTEX-NL4W-0009-SM-5JK3O, CallData(GT=0/0, AD=[29, 0], DP=29, GQ=63, PL=[0, 63, 1041])), Call(sample=GTEX-NPJ7-0009-SM-5JK4M, CallData(GT=0/1, AD=[14, 18], DP=32, GQ=99, PL=[541, 0, 393])), 

In [3]:
#numpy troubles: in terminal, in RIGHT env: conda install numpy==1.15
import numpy


In [20]:
!head /s/project/avsec/ExPecto/resources/geneanno.csv

import pandas as pd
testCSV=pd.read_csv("/s/project/avsec/ExPecto/resources/geneanno.csv")

id,symbol,seqnames,strand,TSS,CAGE_representative_TSS,type
ENSG00000000003,TSPAN6,chrX,-,99894988,99891748,protein_coding
ENSG00000000005,TNMD,chrX,+,99839799,99839933,protein_coding
ENSG00000000419,DPM1,chr20,-,49575092,49575069,protein_coding
ENSG00000000457,SCYL3,chr1,-,169863408,169863037,protein_coding
ENSG00000000460,C1orf112,chr1,+,169631245,169764186,protein_coding
ENSG00000000938,FGR,chr1,-,27961788,27961654,protein_coding
ENSG00000000971,CFH,chr1,+,196621008,196621174,protein_coding
ENSG00000001036,FUCA2,chr6,-,143832827,143832857,protein_coding
ENSG00000001084,GCLC,chr6,-,53481768,53409899,protein_coding


In [26]:
testCSV["id"].values

array(['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419', ...,
       'ENSG00000259763', 'ENSG00000259772', 'ENSG00000259774'],
      dtype=object)

# helper classes for TSS-loader

In [87]:
class rangeInterval(object): #renamed to avoid conflict with genomic interval
    """
    stores window-range-info
    as usual in python, an [ ) interval"""
    def __init__(self, n_up,n_down,anchor=None):
        if n_up==n_down:
            raise AttributeError("An interval without elements is not supported. Your upper and lower bound(exclusive) are the same.")
            
        
        self.lower=n_up
        self.upper=n_down
        self.anchor=anchor
        if anchor is None:
            self.anchor=int((self.lower+self.upper)/2)
    def setAnchor(self,newAnchor):
        self.lower=newAnchor-(self.anchor-self.lower)
        self.upper=newAnchor+(self.upper-self.anchor)
        self.anchor=newAnchor
    def __str__(self):
        return "[{},{},{})".format(self.lower,self.anchor,self.upper)
    def __repr__(self):
        return str(self)
    def __len__(self):
        return self.upper-self.lower
        
test=rangeInterval(4,6)
print(test)
test.setAnchor(0)
test

[4,5,6)


[-1,0,1)

In [71]:
class VcfFastaDummy(object):
    def __init__(self,a,b):
        pass
    def extract(self,interval,sample): 
        allele1="A"*len(interval)
        allele2="C"*len(interval)
        return [allele1,allele2]
    
test=VcfFastaDummy("a","b")
test.extract(Interval(3,5),"testSample")

['AA', 'CC']

In [1]:
! conda install pytorch -y # for dataloader/set-class

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.6.7

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/modules/i12g/anaconda/3-5.0.1/envs/rep

  added / updated specs: 
    - pytorch


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cudnn-7.3.1                |        cuda9.0_0       331.3 MB

The following NEW packages will be INSTALLED:

    cffi:        1.11.5-py36he75722e_1
    cudatoolkit: 9.0-h13b8566_0       
    cudnn:       7.3.1-cuda9.0_0      
    nccl:        1.3.5-cuda9.0_0      
    ninja:       1.8.2-py36h6bb024c_1 
    pycparser:   2.19-py36_0          
    pytorch:     0.4.1-py36ha74772b_0 


cudnn-7.3.1          | 331.3 MB  | ##################################### | 100% 
Preparing transaction: done
Verifying transacti

In [88]:
        from kipoiseq.transforms import ReorderedOneHot
        testTransform=ReorderedOneHot()
        testTransform("ACCC")

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])

# Working with new kipoiseq.extractor modules, install and so on..

In [6]:
vcfFile="/s/project/gtex-processed/vcf/GTEx_Analysis_20150112_WholeGenomeSeq_148Indiv_GATK_HaplotypeCaller.vcf.gz"
vcfTest="vcfTest"
! zcat /s/project/gtex-processed/vcf/GTEx_Analysis_20150112_WholeGenomeSeq_148Indiv_GATK_HaplotypeCaller.vcf.gz | head -n 300 >test.vcf
#add "chr" to each line beginning with a number ( is annotated without "chr", only number..)
!sed 's/^\([0-9]\{1,2\}\t\)/chr\1/' -i test.vcf # NO \t in replacement for some reason, seems to be there anyway
!bgzip -c test.vcf > test.vcf.gz
#create index
!tabix -p vcf test.vcf.gz  


gzip: stdout: Broken pipe


Only usable with index!
(base) [reinharj@ouga04 bazenji]$ bgzip -c vcfTest > test.vcf.gz
(base) [reinharj@ouga04 bazenji]$ tabix -p vcf test.vcf.gz 


In [ ]:
!pip install -U kipoiseq

In [186]:
from importlib import reload
reload(kipoiseq)
reload(kipoiseq.extractors)



<module 'kipoiseq.extractors' from '/opt/modules/i12g/anaconda/3-5.0.1/envs/rep/lib/python3.6/site-packages/kipoiseq/extractors/__init__.py'>

In [ ]:
!conda install -c bioconda cyvcf2 -y #did this manually...

In [89]:
import cyvcf2

In [90]:
from kipoiseq.extractors.vcf_seq import MultiSampleVCF
vcfReader=MultiSampleVCF("test.vcf.gz")

In [91]:
        from pybedtools import Interval
        #"1 for chr1" in this vcf-file....
        testInterval= Interval("chr1", start=10000, end=30000, strand="+", otherfields=None)

In [92]:
from pybedtools import Interval
from kipoiseq.extractors.vcf_seq import MultiSampleVCF

sample1="GTEX-QV31-0003-SM-5URDH"
sample2="GTEX-R55E-0003-SM-5URBX"

vcfReader=MultiSampleVCF("test.vcf.gz")
#"1 for chr1" in this vcf-file....
testInterval= Interval("chr1", start=10175, end=10179, strand="+", otherfields=None) #end=10145 works, 10146 hangs -> first variant there...
testVariant=list(vcfReader.fetch_variants(testInterval,"GTEX-QV31-0003-SM-5URDH"))[0]

In [93]:
testVariant

Variant(chr1:10177 A/C)

In [ ]:
testVariant.genotypes #TODO .genotype or .gt_types?

In [36]:
testVariant.gt_types

array([2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 2,
       0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 2, 2, 0,
       2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2,
       1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2,
       0, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2], dtype=int32)

In [94]:
from kipoiseq.extractors import MultiSampleVCF
class AllelicMultiSampleVCF(MultiSampleVCF):
    """supports variant-query according to genotype
    #TODO clean up code repetition... all in one method...
    """
    def _has_variant_homo(self, variant, sample_id):
        return variant.gt_types[self.sample_mapping[sample_id]] == 2 #might have to use ==3 since 2 is "unknown/*" ?
    def _has_variant_hetero(self, variant, sample_id):
        return variant.gt_types[self.sample_mapping[sample_id]] == 1
    
    def fetch_variants_homo(self, interval, sample_id=None):
        for v in self(self._region(interval)):
            if sample_id is None or self._has_variant_homo(v, sample_id):
                yield v
    def fetch_variants_hetero(self, interval, sample_id=None):
        for v in self(self._region(interval)):
            if sample_id is None or self._has_variant_hetero(v, sample_id):
                yield v      
    

from kipoiseq.extractors import BaseExtractor,VariantSeqExtractor
import pdb
class AllelicVCFSeqExtractor(BaseExtractor):
    def __init__(self, fasta_file, vcf_file):
        self.fasta_file = fasta_file
        self.vcf_file = vcf_file
        self.variant_extractor = VariantSeqExtractor(fasta_file)
        self.vcf = AllelicMultiSampleVCF(vcf_file)
  
    def get_allelic_variants(self,homoVars,heteroVars):
        """
        hetero-allelic variants are only used on first allele, homo- on both
        """
        return [homoVars+heteroVars,homoVars]
        
    def extract(self, interval, anchor=None, sample_id=None, fixed_len=True):
        
        print("getting variants...")
        homoVars=list(self.vcf.fetch_variants_homo(interval, sample_id)) #when mutating them, lists are way easier and not more costly than generators
        heteroVars=list(self.vcf.fetch_variants_hetero(interval, sample_id))
        
        print("sorting variants, homo:{}, hetero:{}...".format(len(homoVars),len(heteroVars)))

        vars1,vars2=self.get_allelic_variants(homoVars,heteroVars)

        print("sortED variants, 1st:{}, 2nd:{}...".format(len(vars1),len(vars2)))
        print("getting mutated seq...")
        
        allele1=self.variant_extractor.extract(
                interval, variants=vars1,
                anchor=anchor, fixed_len=fixed_len)        
        allele2=self.variant_extractor.extract(
                interval, variants=vars2,
                anchor=anchor, fixed_len=fixed_len)
        
        return [allele1,allele2]
    


In [95]:
#test AllelicVCFSeqExtractor
from pybedtools import Interval
sample1="GTEX-QV31-0003-SM-5URDH"
sample2="GTEX-R55E-0003-SM-5URBX"
#"1 for chr1" in this vcf-file....
testInterval= Interval("chr1", start=10140, end=10150, strand="+", otherfields=None) 
testExtractor=AllelicVCFSeqExtractor('/s/genomes/human/hg38/hg38.fa',"/data/ouga/home/ag_gagneur/reinharj/REP/rep/notebooks/bazenji/test.vcf.gz")
testOut=testExtractor.extract(testInterval,10143,sample2)
print(testOut)
testOut[0]==testOut[1]

getting variants...
sorting variants, homo:0, hetero:1...
sortED variants, 1st:1, 2nd:0...
getting mutated seq...
['ccctaaAcct', 'ccctaacccc']


False

In [96]:
from pybedtools import Interval
from kipoiseq.extractors import SingleSeqVCFSeqExtractor
sample1="GTEX-QV31-0003-SM-5URDH" #homo-neg at first snp (1_10146_AC_A_b37) 
sample2="GTEX-R55E-0003-SM-5URBX" #hetero at first snp (1:10146)
sample5="GTEX-N7MS-0009-SM-5JK3E" #homo-neg for snp at 1_10177_A_C_b37 (sample 1/2 homo-unknown there)
sample23="GTEX-OXRK-0004-SM-5JK32" #homo-pos for 1_10327_T_C_b37
sampleNo="GTEX-notexisting"
#"1 for chr1" in this vcf-file....
testInterval= Interval("chr1", start=10140, end=10150, strand="+", otherfields=None) 
testExtractor=SingleSeqVCFSeqExtractor('/s/genomes/human/hg38/hg38.fa',"/data/ouga/home/ag_gagneur/reinharj/REP/rep/notebooks/bazenji/test.vcf.gz")
testOut=testExtractor.extract(testInterval,10143,sample23)
print(testOut)
testOut[0]==testOut[1]

ccctaacccc


True

testing pyvcf

/opt/modules/i12g/anaconda/3-5.0.1/envs/rep/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [7]:
from pybedtools import Interval
import cyvcf2
testVCF=cyvcf2.VCF("/data/ouga/home/ag_gagneur/reinharj/REP/rep/notebooks/bazenji/test.vcf.gz")
#testInterval= Interval("chr1", start=10140, end=10146, strand="+", otherfields=None) #end=10145 works, 10146 hangs -> first variant there...
testVariantGen=testVCF("chr1:10220-10233")
#testVariantGen=testVCF("chr1:2-9") #"official" testfile

for v in testVariantGen:
    print(v)

chr1	10230	1_10230_AC_A_b37	AC	A	97.75	LCR	AC=1;AF=0.003704;AN=270;BaseQRankSum=0.747;CCC=2596;ClippingRankSum=-0.747;DP=9628;FS=0;GQ_MEAN=10.21;GQ_STDDEV=15.23;HWP=0.0046;InbreedingCoeff=-0.1778;MQ=31.9;MQ0=0;MQRankSum=0.747;NCC=232;NEGATIVE_TRAIN_SITE;QD=6.98;ReadPosRankSum=-0.747;VQSLOD=0.412;culprit=FS	GT:AD:DP:GQ:PL	0/0:96,0:96:0:0,0,777	./.:64,0:64:.:.	0/0:91,0:91:0:0,0,401	0/0:88,0:88:0:0,0,640	0/0:21,0:21:0:0,0,45	0/0:2,0:2:6:0,6,55	0/0:22,0:22:0:0,0,431	0/0:21,0:21:0:0,0,95	0/0:48,0:48:0:0,0,54	0/0:60,0:60:0:0,0,1314	./.:12,0:12:.:.	0/0:48,0:48:0:0,0,468	0/0:28,0:28:0:0,0,215	0/0:23,0:23:30:0,30,450	./.:25,0:25:.:.	0/0:51,0:51:40:0,40,1392	0/0:66,0:66:0:0,0,708	0/0:27,0:27:0:0,0,317	0/0:64,0:64:0:0,0,434	0/0:40,0:40:0:0,0,478	0/0:65,0:65:0:0,0,1102	0/0:22,0:22:14:0,14,574	0/0:39,0:39:0:0,0,908	0/0:23,0:23:0:0,0,194	0/0:34,0:34:8:0,8,783	0/0:19,0:19:0:0,0,325	0/0:38,0:38:0:0,0,601	0/0:64,0:64:0:0,0,735	0/0:61,0:61:0:0,0,570	0/0:48,0:48:0:0,0,827	0/0:53,0:53:0:0,0,332	0/0:35,0:3

In [ ]:
import cyvcf2

In [4]:
cyvcf2.__version__

'0.10.0'

# Output-writer with Zarr

In [5]:
! pip install zarr

    100% |████████████████████████████████| 3.3MB 1.3MB/s 
    100% |████████████████████████████████| 3.8MB 1.0MB/s 
  Running setup.py bdist_wheel for zarr ... done
  Stored in directory: /data/ouga/home/ag_gagneur/reinharj/.cache/pip/wheels/60/68/03/ed754a88e8af5cc9d77d6b043ac7913a69a0c4bccf036b56b2
  Running setup.py bdist_wheel for asciitree ... done
  Stored in directory: /data/ouga/home/ag_gagneur/reinharj/.cache/pip/wheels/1d/d9/58/9808b306744df0208fccc640d3d9952a5bc7468502d42897d5
  Running setup.py bdist_wheel for numcodecs ... done
  Stored in directory: /data/ouga/home/ag_gagneur/reinharj/.cache/pip/wheels/75/ea/37/8a1c2b6e937606f90dd0a9659783790d922efd1cf98d7a82a9
Successfully built zarr asciitree numcodecs


In [128]:
class ZarrVariantPredWriter(object):
    def __init__(self,outputFile,genes,samples,cAlleles,genePositions,predSize):
        """set up data array, annotate with attributes
        cAlleles and predSize are given as raw numbers/dimension size
        genes, samples and genePositions are given as elements
        
        """
        import zarr
        self.filename=outputFile
        self.cGenes=len(genes)
        self.cSamples=len(samples)
        self.cGenePositions=len(genePositions)
        self.cAlleles=cAlleles
        self.cPredSize=predSize
        #Allele got canceled, just append allele 2 to allele1
        self.data=zarr.open(self.filename, mode='w', shape=(self.cGenes,self.cSamples,self.cAlleles,self.cGenePositions,self.cPredSize),
                 dtype='f8') #TODO: precision right? chunksize left out for now, are there default values? "chunks=(1000, 1000),"
        
        self.data.attrs['genes'] = self.as_idx_dict(genes)
        self.data.attrs["samples"]=self.as_idx_dict(samples)
        self.data.attrs["genePositions"]=self.as_idx_dict(genePositions)
    @staticmethod    
    def as_idx_dict(lis):
        """for O1 lookup"""
        dic={}
        for idx,elem in enumerate(lis):
            dic[elem]=idx
        return dic
        
    def batch_write(self,annotatedPreds):
        """plug in predictions in right position, given by metadata and annotation-array

        gene, sample and allele can differ from prediction to prediction
        expects all locations on gene in one go
        """
        print("....writing batch")
        for inputIdx,predPerInput in enumerate(annotatedPreds["preds"]):
            geneId=annotatedPreds["metadata"]["gene_id"][inputIdx]
            sampleId=annotatedPreds["metadata"]["sample_id"][inputIdx]
            
            alleleIdx=annotatedPreds["metadata"]["allele_id"][inputIdx]
            geneIdx=self.data.attrs["genes"][geneId]
            sampleIdx=self.data.attrs["samples"][sampleId]
            self.data[geneIdx,sampleIdx,alleleIdx]=predPerInput

        return
    def close(self):
        """
        According to https://zarr.readthedocs.io/en/stable/api/creation.html:
        Notes
        There is no need to close an array. Data are automatically flushed to the file system."""
        pass 
        

        
import numpy as np 
test=ZarrVariantPredWriter("test",["gene1","gene2","gene4"],["samp1","samp2"],2,[0],10)
testPreds=np.array([range(0,10),range(10,20)])
testPredsShapedArray=np.reshape(testPreds,(2,1,10))
testBatch={"preds":testPredsShapedArray,
          "metadata":{
              "gene_id":np.array(["gene4","gene2"]),
              "sample_id":np.array(["samp2","samp1"]),
              "allele_id":np.array([1,1])
          }}
test.batch_write(testBatch)

#Todo: read and check...

....writing batch


Use with Async batchwriter for buffering

In [132]:
import kipoi.writers as wrt
wrt.AsyncBatchWriter()

AttributeError: module 'kipoi.writers' has no attribute 'AsyncBatchWriter'

In [144]:
!pip install kipoi -U

  Using cached https://files.pythonhosted.org/packages/61/27/50ce54d62cc4a2696fe62290634f924bd80e54282964a198146e9c1691dc/kipoi-0.6.9-py3-none-any.whl
  Found existing installation: kipoi 0.6.5
    Uninstalling kipoi-0.6.5:
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/data/nasif12/modules_if12/SL7/i12g/anaconda/3-5.0.1/envs/rep/bin/kipoi'
Consider using the `--user` option or check the permissions.



# TSSloader

In [122]:
#data-loader tut here: https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel
from torch.utils.data import Dataset
#formerly "repDataloader"
class RepDataset(Dataset):
    
    from pybedtools import Interval
    
    def __init__(self,refGenFasta,vcfFile,tssFile,n_upstream,n_downstream,sample_ids,gene_ids=None):
        """set instance-vars, read TSS completely, load (dummy) VCFfastaExtractor"""

        import pandas as pd

        from kipoiseq.transforms import ReorderedOneHot
        self.transform=ReorderedOneHot()
        
        
        self.vcfFile=vcfFile
        self.refGenFasta=refGenFasta
        self.varExtractor=None
        #AllelicVCFSeqExtractor(self.refGenFasta,vcfFile)#VcfFastaDummy(vcfFile,refGenFasta)
        
        self.sampleIDs=sample_ids
        self.interval=None
        self.refInterval=rangeInterval(n_upstream,n_downstream)
        
        
        #TODO use only prot-coding? Separate file or bound to schema of example file?
        self.tssCollection=pd.read_csv(tssFile) # needs field "TSS" with RegGen position and "strand" with "+" or "-"
        if gene_ids:
            self.geneIDs= gene_ids
            tssCollection=tssCollection.loc[tssCollection['id'].isin( gene_ids)]
            self.geneIDs=self.tssCollection["id"].values #maybe specified IDs not in file...
        else:
            self.geneIDs=self.tssCollection["id"].values
        


    def __len__(self):
        return len(self.sampleIDs)*len(self.geneIDs)
    def __getitem__(self,idx):
        import numpy as np
        import copy
        """transform idx to sample- and tss-id, then get both alleles from extractor"""
        
        #each worker needs own file handle:
        if self.varExtractor==None:
            self.varExtractor=AllelicVCFSeqExtractor(self.refGenFasta,self.vcfFile)
        if self.interval==None:
            self.interval=copy.deepcopy(self.refInterval)
        
        sampleIdx=int(idx/len(self.tssCollection))
        tssIdx=int(idx%len(self.tssCollection))
        
        
        chromo=self.tssCollection.loc[tssIdx,"seqnames"]
        strand=self.tssCollection.loc[tssIdx,"strand"]
        tssStart=self.tssCollection.loc[tssIdx,"TSS"]
        
        self.interval.setAnchor(tssStart)
        genomic_interval=self.getGenomicInterval(chromo,strand)  
        alleles= self.varExtractor.extract(genomic_interval,self.interval.anchor,self.sampleIDs[sampleIdx]) #strand-info where?
        
        #formatting output
        gene_id=self.tssCollection.loc[tssIdx,"id"]
        sample_id=self.sampleIDs[sampleIdx]
        returnBatch={}
        returnBatch["inputs"]=[self.transform(alleles[0].upper()),self.transform(alleles[1].upper())]
        #need np.array-conversion, or it will concat on other dimension for batchsize>1
        returnBatch["metadata"]={"gene_id":np.array([gene_id]*2),"sample_id":np.array([sample_id]*2),"allele_id":np.array([0,1])} 
        return returnBatch
    
    def getGenomicInterval(self,chromosome,strand):
        """converts internal interval to bedInterval as used in extractor"""
        from pybedtools import Interval
        
        return Interval(chromosome, start=self.interval.lower, end=self.interval.upper, strand=strand, otherfields=None)
    def get_gene_ids(self):
        return self.geneIDs
        

# actual pipeline

In [79]:
#make test-tss file
#! head -n 20 /s/project/avsec/ExPecto/resources/geneanno.csv> testTSS.csv
tssFile="testTSS.csv"

In [110]:
#params
basenjiSeqSize=131072 
tssFile="testTSS.csv"
refGen='/s/genomes/human/hg38/hg38.fa'
vcfFile="/data/ouga/home/ag_gagneur/reinharj/REP/rep/notebooks/bazenji/test.vcf.gz"
outfile="test.zarr"

sample1="GTEX-QV31-0003-SM-5URDH" #homo-neg at first snp (1_10146_AC_A_b37) 
sample2="GTEX-R55E-0003-SM-5URBX" #hetero at first snp (1:10146)
sample5="GTEX-N7MS-0009-SM-5JK3E" #homo-neg for snp at 1_10177_A_C_b37 (sample 1/2 homo-unknown there)
sample23="GTEX-OXRK-0004-SM-5JK32" #homo-pos for 1_10327_T_C_b37
samples=[sample1,sample2,sample5,sample23]

cAlleles=2
predSize=4229
binsOnSeq=[int(960/2)] #must be continous!

In [123]:
#test single batch
testData = RepDataset(refGen,vcfFile,tssFile,-int(basenjiSeqSize/2),int(basenjiSeqSize/2),samples)
geneIDs= testData.get_gene_ids()
testBatch=testData.__getitem__(1)

getting variants...
sorting variants, homo:0, hetero:0...
sortED variants, 1st:0, 2nd:0...
getting mutated seq...


no intervals found for b'/data/ouga/home/ag_gagneur/reinharj/REP/rep/notebooks/bazenji/test.vcf.gz' at chrX:99774263-99905335
no intervals found for b'/data/ouga/home/ag_gagneur/reinharj/REP/rep/notebooks/bazenji/test.vcf.gz' at chrX:99774263-99905335


In [70]:
testData.get_gene_ids()

array(['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419',
       'ENSG00000000457', 'ENSG00000000460', 'ENSG00000000938',
       'ENSG00000000971', 'ENSG00000001036', 'ENSG00000001084',
       'ENSG00000001167', 'ENSG00000001460', 'ENSG00000001461',
       'ENSG00000001497', 'ENSG00000001561', 'ENSG00000001617',
       'ENSG00000001626', 'ENSG00000001629', 'ENSG00000001630',
       'ENSG00000001631'], dtype=object)

In [106]:
!pip install biopython==1.73

  Using cached https://files.pythonhosted.org/packages/28/15/8ac646ff24cfa2588b4d5e5ea51e8d13f3d35806bd9498fbf40ef79026fd/biopython-1.73-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: biopython 1.72
    Uninstalling biopython-1.72:
      Successfully uninstalled biopython-1.72


In [129]:
##MAIN routine!
def pred_and_store():
    inputData = RepDataset(refGen,vcfFile,tssFile,-int(basenjiSeqSize/2),int(basenjiSeqSize/2),samples)
    geneIDs= inputData.get_gene_ids()
    
    from torch.utils import data
    from kipoi.data_utils import numpy_collate_concat
    generator = data.DataLoader(inputData,batch_size=1, collate_fn=numpy_collate_concat)

    import kipoi
    model = kipoi.get_model('Basenji')



    writer=ZarrVariantPredWriter(outfile,geneIDs,samples,cAlleles,binsOnSeq,predSize)


    for batch in generator:
        preds=model.predict_on_batch(batch['inputs'])
        #TODO choose/compute right bin AFTER TSS, not over. For now, just use middle
        #see https://github.com/kipoi/models/issues/87 for info n dimensions

        #remember: id_inputvec(batchsize,here allele1 and 2), id_genomicInterval, id_features/"tissues"
        tssExpr=preds[:,binsOnSeq[0]:(binsOnSeq[-1]+1),:]
        batch["preds"]=tssExpr
        writer.batch_write(batch)

    writer.close()
pred_and_store()

Using downloaded and verified file: /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/ckp/model.meta
Using downloaded and verified file: /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/ckp/model.index
Using downloaded and verified file: /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/ckp/model.data-00000-of-00001
Using downloaded and verified file: /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/const_feed_dict_pkl/3a76c37eb9ad255680ba774b110de1be
INFO:tensorflow:Restoring parameters from /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/ckp/model
getting variants...
sorting variants, homo:0, hetero:0...
sortED variants, 1st:0, 2nd:0...
getting mutated seq...


no intervals found for b'/data/ouga/home/ag_gagneur/reinharj/REP/rep/notebooks/bazenji/test.vcf.gz' at chrX:99829452-99960524
no intervals found for b'/data/ouga/home/ag_gagneur/reinharj/REP/rep/notebooks/bazenji/test.vcf.gz' at chrX:99829452-99960524


....writing batch
getting variants...
sorting variants, homo:0, hetero:0...
sortED variants, 1st:0, 2nd:0...
getting mutated seq...


no intervals found for b'/data/ouga/home/ag_gagneur/reinharj/REP/rep/notebooks/bazenji/test.vcf.gz' at chrX:99774263-99905335
no intervals found for b'/data/ouga/home/ag_gagneur/reinharj/REP/rep/notebooks/bazenji/test.vcf.gz' at chrX:99774263-99905335


....writing batch
getting variants...
sorting variants, homo:0, hetero:0...
sortED variants, 1st:0, 2nd:0...
getting mutated seq...


no intervals found for b'/data/ouga/home/ag_gagneur/reinharj/REP/rep/notebooks/bazenji/test.vcf.gz' at chr20:49509556-49640628
no intervals found for b'/data/ouga/home/ag_gagneur/reinharj/REP/rep/notebooks/bazenji/test.vcf.gz' at chr20:49509556-49640628


....writing batch
getting variants...
sorting variants, homo:0, hetero:0...
sortED variants, 1st:0, 2nd:0...
getting mutated seq...
....writing batch
getting variants...
sorting variants, homo:0, hetero:0...
sortED variants, 1st:0, 2nd:0...
getting mutated seq...


KeyboardInterrupt: 

In [41]:
import kipoi
model = kipoi.get_model('Basenji')
preds=model.predict_on_batch(testBatch['inputs'])

Using downloaded and verified file: /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/ckp/model.meta
Using downloaded and verified file: /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/ckp/model.index
Using downloaded and verified file: /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/ckp/model.data-00000-of-00001
Using downloaded and verified file: /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/const_feed_dict_pkl/3a76c37eb9ad255680ba774b110de1be
INFO:tensorflow:Restoring parameters from /data/nasif12/home_if12/reinharj/.kipoi/models/Basenji/downloaded/model_files/ckp/model


In [141]:
preds.shape
#TODO choose/compute right bin AFTER TSS, not over. For now, just use middle
#see https://github.com/kipoi/models/issues/87 for info n dimensions
tssExpr=preds[:,int(960/2),:]

In [67]:
! conda env list

# conda environments:
#
base                     /opt/modules/i12g/anaconda/3-5.0.1
DeepBind                 /opt/modules/i12g/anaconda/3-5.0.1/envs/DeepBind
DeepSEA                  /opt/modules/i12g/anaconda/3-5.0.1/envs/DeepSEA
FactorNet                /opt/modules/i12g/anaconda/3-5.0.1/envs/FactorNet
Python3.5Test            /opt/modules/i12g/anaconda/3-5.0.1/envs/Python3.5Test
RL                       /opt/modules/i12g/anaconda/3-5.0.1/envs/RL
avsec                    /opt/modules/i12g/anaconda/3-5.0.1/envs/avsec
cadd-env                 /opt/modules/i12g/anaconda/3-5.0.1/envs/cadd-env
cadd-env-py36            /opt/modules/i12g/anaconda/3-5.0.1/envs/cadd-env-py36
cadd13_py27              /opt/modules/i12g/anaconda/3-5.0.1/envs/cadd13_py27
cadd_temp                /opt/modules/i12g/anaconda/3-5.0.1/envs/cadd_temp
celikm-mtsplice          /opt/modules/i12g/anaconda/3-5.0.1/envs/celikm-mtsplice
chipnexus                /opt/modules/i12g/anaconda/3-5.0.1/envs/chipnexus
deep-rbp       

In [69]:
! conda env export >~/REP/bazenji-pred.yml #also change first line "name" from rep to bazenji-pred

In [ ]:
conda env create -f REP/bazenji-pred.yml 
:
ResolvePackageNotFound: 
  - notebook==5.7.4=py36_1000
  - jupyter_client==5.2.4=py_0
  - nbconvert==5.3.1=py_1
->removed those...